In [ ]:
# Write your imports here
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

import os
from PIL import Image
import imagehash

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import glob

import tensorflow as tf

# Working with Images Lab
## Information retrieval, preprocessing, and feature extraction

In this lab, you'll work with images of felines (cats), which have been classified according to their taxonomy. Each subfolder contains images of a particular species. The dataset is located [here](https://www.kaggle.com/datasets/datahmifitb/felis-taxonomy-image-classification) but it's also provided to you in the `data/` folder.

### Problem 1. Some exploration (1 point)
How many types of cats are there? How many images do we have of each? What is a typical image size? Are there any outliers in size?

First, we are going to read the data and store the needed information in variables for species, number and size of the images. 

In [ ]:
data_folder = 'data'

species = []
num_images = []
image_sizes = []

for subfolder in os.listdir(data_folder):
    subfolder_path = os.path.join(data_folder, subfolder)
    if os.path.isdir(subfolder_path):
        # List images in the subfolder
        images = [img for img in os.listdir(subfolder_path) if img.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif'))]
        
        # Store species and number of images
        species.append(subfolder)
        num_images.append(len(images))
        
        # Analyze image sizes
        for image_name in images:
            image_path = os.path.join(subfolder_path, image_name)
            try:
                with Image.open(image_path) as img:
                    image_sizes.append(img.size)
            except Exception as e:
                print(f"Error opening image {image_path}: {e}")

Second, we are going to create a dataframe with image height and width and then calculate the typical image size using mean and median. In order to analyze the size and detect any potential outliers, we are going to use the $IQR$ method, which focuses on the middle 50% of the data. The Interquartile Range is the range between the first quartile ($Q1$) and the third quartile ($Q3$) by the formula $IQR = Q3 - Q1$. The outliers are typically defined as data points which fall below $Q1 - 1.5 * IQR$ or above $Q3 + 1.5 * IQR$.

In [ ]:
sizes_df = pd.DataFrame(image_sizes, columns=['Width', 'Height'])

# Calculate typical image size
mean_size = sizes_df.mean()
median_size = sizes_df.median()

# Detect outliers using IQR method
Q1 = sizes_df.quantile(0.25)
Q3 = sizes_df.quantile(0.75)
IQR = Q3 - Q1
outliers = sizes_df[(sizes_df < (Q1 - 1.5 * IQR)) | (sizes_df > (Q3 + 1.5 * IQR))]

# Output the results
species_images_count = dict(zip(species, num_images))
typical_size = (mean_size, median_size)
outliers_count = outliers.dropna().shape[0]


In [ ]:
for k, v in species_images_count.items():
    print(f'{k}: {v}')

print(f'Mean image size: Width = {mean_size["Width"]:.2f}, Height = {mean_size["Height"]:.2f}')
print(f'Median image size: Width = {median_size["Width"]:.1f}, Height = {median_size["Height"]:.1f}')

print(f'Number of outliers: {outliers_count}.')

### Problem 2. Duplicat(e)s (1 point)
Find a way to filter out (remove) identical images. I would recommnend using file hashes, but there are many approaches. Keep in mind that during file saving, recompression, etc., a lot of artifacts can change the file content (bytes), but not visually.

We are going to need a set to store the unique image hashes and traverse the data folder in order to go through all of the images to check for duplicates, which in fact is quite straight-forward using hashes.

In [ ]:
species = []
num_images = []
image_hashes = set()
removed_images = 0

# Traverse the data folder
for subfolder in os.listdir(data_folder):
    subfolder_path = os.path.join(data_folder, subfolder)
    if os.path.isdir(subfolder_path):
        images = [img for img in os.listdir(subfolder_path) if img.lower().endswith(('png', 'jpg', 'jpeg', 'tiff', 'bmp', 'gif'))]
        
        species.append(subfolder)
        unique_images = 0
        
        # Analyze image hashes
        for image_name in images:
            image_path = os.path.join(subfolder_path, image_name)
            try:
                with Image.open(image_path) as img:
                    img_hash = imagehash.average_hash(img)
                    if img_hash not in image_hashes:
                        image_hashes.add(img_hash)
                        unique_images += 1
                    else:
                        removed_images += 1
            except Exception as e:
                print(f"Error opening image {image_path}: {e}")
        
        num_images.append(unique_images)

species_images_count = dict(zip(species, num_images))

for specie, count in species_images_count.items():
    print(f'Species: {specie}, Number of unique images: {count}')

print(f'Number of removed images: {removed_images}')

### Problem 3. Loading a model (2 points)
Find a suitable, trained convolutional neural network classifier. I recommend `ResNet50` as it's small enough to run well on any machine and powerful enough to make reasonable predictions. Most ready-made classifiers have been trained for 1000 classes.

You'll need to install libraries and possibly tinker with configurations for this task. When you're done, display the total number of layers and the total number of parameters. For ResNet50, you should expect around 50 layers and 25M parameters.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

In [ ]:
model = ResNet50(weights='imagenet')

model.summary()

total_layers = len(model.layers)
total_params = model.count_params()

print(f'Total number of layers: {total_layers}')
print(f'Total number of parameters: {total_params:,}')

### Problem 4. Prepare the images (1 point)
You'll need to prepare the images for passing to the model. To do so, they have to be resized to the same dimensions. Most available models have a specific requirement for sizes. You may need to do additional preprocessing, depending on the model requirements. These requirements should be easily available in the model documentation.

In [ ]:
model = ResNet50(weights='imagenet')

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def preprocess_images_in_directory(directory_path):
    img_paths = glob.glob(os.path.join(directory_path, '**', '*.jpg'), recursive=True)
    preprocessed_images = []

    if not img_paths:
        print("No images found in the directory.")
        return np.array([])

    for img_path in img_paths:
        preprocessed_img = preprocess_image(img_path)
        if preprocessed_img is not None:
            preprocessed_images.append(preprocessed_img)

    if preprocessed_images:
        batch = np.vstack(preprocessed_images)
        return batch
    else:
        print("No valid images to preprocess.")
        return np.array([])

directory_path = 'data'
preprocessed_batch = preprocess_images_in_directory(directory_path)

predictions = model.predict(preprocessed_batch)
print(predictions)

if preprocessed_batch.size > 0:
    # Now the batch is ready to be passed to the model for prediction or feature extraction
    predictions = model.predict(preprocessed_batch)
    print(predictions)
else:
    print("No images to process.")



In order to understand the predictions, we will need to decode them, which would provide us with each image's highest probabilities for appended classes. 

In [ ]:
from tensorflow.keras.applications.resnet50 import decode_predictions

decoded_predictions = decode_predictions(predictions, top=5)

for i, prediction in enumerate(decoded_predictions):
    print(f"Image {i+1} predictions:")
    for pred in prediction:
        print(f" - {pred[1]}: {pred[2]*100:.2f}%")

### Problem 5. Load the images efficiently (1 point)
Now that you've seen how to prepare the images for passing to the model... find a way to do it efficiently. Instead of loading the entire dataset in the RAM, read the images in batches (e.g. 4 images at a time). The goal is to read these, preprocess them, maybe save the preprocessed results in RAM.

If you've already done this in one of the previous problems, just skip this one. You'll get your point for it.

\* Even better, save the preprocessed image arrays (they will not be valid .jpg file) as separate files, so you can load them "lazily" in the following steps. This is a very common optimization to work with large datasets.

As we saw from the previous problem, predictions can take quite a while if not done with consideration to memory, so next we are going to implement a lazy method of reading and preprocessing them. We are also going to set the code up to properly handle the next problems, such as confusion matrix and predictions.

In [ ]:
# Create a mapping from class names to numeric labels
class_names = sorted([d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))])
class_to_label = {class_name: idx for idx, class_name in enumerate(class_names)}

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    return img_array

# Function to get true labels from the directory structure
def get_true_labels(directory_path, class_to_label):
    true_labels = []
    img_paths = glob.glob(os.path.join(directory_path, '**', '*.jpg'), recursive=True) + \
                glob.glob(os.path.join(directory_path, '**', '*.png'), recursive=True)

    for img_path in img_paths:
        class_name = os.path.basename(os.path.dirname(img_path))
        label = class_to_label.get(class_name)
        if label is not None:
            true_labels.append(label)
        else:
            print(f"Warning: Class name '{class_name}' not found in class_to_label mapping.")

    return true_labels

# Get true labels
true_labels = get_true_labels(directory_path, class_to_label)

# Function to save a batch of preprocessed images
def save_preprocessed_batch(batch, batch_index, save_dir='preprocessed_batches'):
    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, f'batch_{batch_index}.npy'), batch)

# Function to load and preprocess images in batches
def preprocess_images_in_directory(directory_path, batch_size=4):
    img_paths = glob.glob(os.path.join(directory_path, '**', '*.jpg'), recursive=True) + \
                glob.glob(os.path.join(directory_path, '**', '*.png'), recursive=True)
    preprocessed_images = []
    batch_index = 0

    if not img_paths:
        print("No images found in the directory.")
        return

    for i, img_path in enumerate(img_paths):
        preprocessed_img = preprocess_image(img_path)
        preprocessed_images.append(preprocessed_img)

        if len(preprocessed_images) == batch_size:
            batch = np.stack(preprocessed_images)
            save_preprocessed_batch(batch, batch_index)
            preprocessed_images = []
            batch_index += 1

    if preprocessed_images:  # Save the last batch if it's not empty
        batch = np.stack(preprocessed_images)
        save_preprocessed_batch(batch, batch_index)

    print(f"Processed and saved {batch_index + 1} batches.")

# Function to save a batch of preprocessed images
def save_preprocessed_batch(batch, batch_index, save_dir='preprocessed_batches'):
    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, f'batch_{batch_index}.npy'), batch)

# Preprocess images and save batches to disk
preprocess_images_in_directory(directory_path, batch_size=4)

# Function to load a preprocessed batch from disk
def load_preprocessed_batch(batch_index, save_dir='preprocessed_batches'):
    batch_path = os.path.join(save_dir, f'batch_{batch_index}.npy')
    if os.path.exists(batch_path):
        return np.load(batch_path)
    else:
        return None

### Problem 6. Predictions (1 point)
Finally, you're ready to get into the meat of the problem. Obtain predictions from your model and evaluate them. This will likely involve manual work to decide how the returned classes relate to the original ones.

Create a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) to evaluate the classification.

In [ ]:
# Function to make predictions
def make_predictions(num_batches, save_dir='preprocessed_batches'):
    all_predictions = []

    for batch_index in range(num_batches):
        batch = load_preprocessed_batch(batch_index, save_dir)

        if batch is not None:
            predictions = model.predict(batch)
            all_predictions.extend(predictions)

    return all_predictions

# Making predictions
num_batches = len(os.listdir('preprocessed_batches'))
predictions = make_predictions(num_batches)

# Convert predictions to labels
predicted_labels = np.argmax(np.array(predictions), axis=1)

# Convert true_labels to numpy array
true_labels_np = np.array(true_labels)

# Ensure that true_labels and predicted_labels have the same length
assert len(true_labels_np) == len(predicted_labels), "Mismatch between true labels and predicted labels"

# Check the shape of predictions
for i, batch_predictions in enumerate(predictions):
    print(f"Batch {i + 1} predictions shape: {batch_predictions.shape}")

Let's now incorporate a confusion matrix in our predictions, which will show us how well our model is performing, providing a detailed view of the classification accuracy across all classes.

In [ ]:
# Assuming predictions are being converted to labels correctly
predicted_labels_np = np.array(predicted_labels)
true_labels_np = np.array(true_labels)

def get_class_names(directory_path):
    class_names = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    return sorted(class_names)

directory_path = 'data'
class_names = get_class_names(directory_path)

num_classes = len(class_names)

predicted_labels_np = np.clip(predicted_labels_np, 0, num_classes - 1)

# Compute confusion matrix
cm = tf.math.confusion_matrix(true_labels_np, predicted_labels_np, num_classes=num_classes)
cm_np = cm.numpy()

# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm_np, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.colorbar()

tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, np.arange(num_classes), rotation=45)
plt.yticks(tick_marks, np.arange(num_classes))

for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, int(cm_np[i, j]), ha='center', va='center', color='white' if cm_np[i, j] > cm_np.max() / 2 else 'black')

plt.show()

### Problem 7. Grayscale (1 point)
Converting the images to grayscale should affect the classification negatively, as we lose some of the color information.

Find a way to preprocess the images to grayscale (using what you already have in Problem 4 and 5), pass them to the model, and compare the classification results to the previous ones.

In [ ]:
def preprocess_image_grayscale(img_path):
    img = Image.open(img_path).convert('L')
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=-1)
    img_array = np.repeat(img_array, 3, axis=-1)
    img_array = preprocess_input(img_array)
    return img_array

def preprocess_images_in_directory_grayscale(directory_path, batch_size=4):
    img_paths = glob.glob(os.path.join(directory_path, '**', '*.jpg'), recursive=True) + \
                glob.glob(os.path.join(directory_path, '**', '*.png'), recursive=True)
    preprocessed_images = []
    batch_index = 0

    if not img_paths:
        print("No images found in the directory.")
        return

    for i, img_path in enumerate(img_paths):
        preprocessed_img = preprocess_image_grayscale(img_path)
        preprocessed_images.append(preprocessed_img)

        if len(preprocessed_images) == batch_size:
            batch = np.stack(preprocessed_images)
            save_preprocessed_batch(batch, batch_index, save_dir='grayscale_batches')
            preprocessed_images = []
            batch_index += 1

    if preprocessed_images:  # Save the last batch if it's not empty
        batch = np.stack(preprocessed_images)
        save_preprocessed_batch(batch, batch_index, save_dir='grayscale_batches')

    print(f"Processed and saved {batch_index + 1} grayscale batches.")

# Preprocess grayscale images and save batches to disk
preprocess_images_in_directory_grayscale(directory_path, batch_size=4)

def make_predictions_grayscale(num_batches, save_dir='grayscale_batches'):
    all_predictions = []

    for batch_index in range(num_batches):
        batch = load_preprocessed_batch(batch_index, save_dir)

        if batch is not None:
            predictions = model.predict(batch)
            all_predictions.extend(predictions)

    return all_predictions

# Number of grayscale batches
num_batches_grayscale = len(os.listdir('grayscale_batches'))

# Making predictions with grayscale images
predictions_grayscale = make_predictions_grayscale(num_batches_grayscale)

# Ensure predictions and true labels are numpy arrays of integer type
true_labels_np = np.array(true_labels).astype(int)
predicted_labels_grayscale = np.array(predicted_labels_grayscale).astype(int)

# Clip predicted labels to ensure they are within the valid range
predicted_labels_grayscale = np.clip(predicted_labels_grayscale, 0, num_classes - 1)

# Check shapes and sample values
print("Shape of true_labels_np:", true_labels_np.shape)
print("Shape of predicted_labels_grayscale:", predicted_labels_grayscale.shape)
print("Sample values from true_labels_np:", true_labels_np[:10])
print("Sample values from predicted_labels_grayscale:", predicted_labels_grayscale[:10])

# Compute confusion matrix
cm_grayscale = tf.math.confusion_matrix(true_labels_np, predicted_labels_grayscale, num_classes=num_classes)
cm_np_grayscale = cm_grayscale.numpy()

# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm_np_grayscale, cmap=plt.cm.Blues)
plt.title('Confusion Matrix (Grayscale)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.colorbar()

tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, np.arange(num_classes), rotation=45)
plt.yticks(tick_marks, np.arange(num_classes))

for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, int(cm_np_grayscale[i, j]), ha='center', va='center', color='white' if cm_np_grayscale[i, j] > cm_np_grayscale.max() / 2 else 'black')

plt.show()

### Problem 8. Deep image features (1 point)
Find a way to extract one-dimensional vectors (features) for each (non-grayscale) image, using your model. This is typically done by "short-circuiting" the model output to be an intermediate layer, while keeping the input the same. 

In case the outputs (also called feature maps) have different shapes, you can flatten them in different ways. Try to not create huge vectors; the goal is to have a relatively short sequence of numbers which describes each image.

You may find a tutorial like [this](https://towardsdatascience.com/exploring-feature-extraction-with-cnns-345125cefc9a) pretty useful but note your implementation will depend on what model (and framework) you've decided to use.

It's a good idea to save these as one or more files, so you'll spare yourself a ton of preprocessing.

### Problem 9. Putting deep image features to use (1 points)
Try to find similar images, using a similarity metric on the features you got in the previous problem. Two good metrics are `mean squared error` and `cosine similarity`. How do they work? Can you spot images that look too similar? Can you explain why?

\* If we were to take Fourier features (in a similar manner, these should be a vector of about the same length), how do they compare to the deep features; i.e., which features are better to "catch" similar images?

### * Problem 10. Explore, predict, and evaluate further
You can do a ton of things here, at your desire. For example, how does masking different areas of the image affect classification - a method known as **saliency map** ([info](https://en.wikipedia.org/wiki/Saliency_map))? Can we detect objects? Can we significantly reduce the number of features (keeping the quality) that we get? Can we reliably train a model to predict our own classes? We'll look into these in detail in the future.